In [1]:
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense,LSTM,Embedding
from keras.preprocessing.sequence import pad_sequences
df=pd.read_csv(r"B:\project\newdataset\chunk_2.csv")
df = df.drop(columns=['Unnamed: 0'])
df

,text,generated
0,The use of cars has always affected society gr...,0.0
1,Cars are a part of everyday life. You use them...,0.0
2,"In German Suburb, Life Goes On Without Cars. S...",0.0
3,Do you drive a car? A truck? Or any kind of ve...,0.0
4,Cars are one of the most iconic things about A...,0.0
...,...,...
17983,"By the time a student turns 18, they have spen...",0.0
17984,Online schooling or video conference? I suppor...,0.0
17985,"In America, students are far from the top perf...",0.0
17986,"Imagine for a minute that you are at school, o...",0.0


In [2]:
from transformers import LongformerTokenizer
tokenizer=LongformerTokenizer.from_pretrained('allenai/longformer-base-4096')
essays=df['text'].tolist()
tokens=tokenizer(essays,return_tensors='pt',padding=True,truncation=True)

In [3]:
x=tokens['input_ids']
y=df['generated']
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
input_ids_size = tokens['input_ids'].size()
attention_mask_size = tokens['attention_mask'].size()
print("Input IDs size:", input_ids_size)
print("Attention Mask size:", attention_mask_size)
# Assuming 'tokens' is the result of tokenizer
unique_tokens = set(token.item() for seq in tokens['input_ids'] for token in seq)
vocabulary_size = len(unique_tokens)

print("Vocabulary Size:", vocabulary_size)
# Assuming 'input_ids' is the first element in xtrain
max_sequence_length_x = max(len(seq) for seq in xtrain)

# Print or use the max_sequence_length_x in your pad_sequences and Embedding layer
print("Max Sequence Length:", max_sequence_length_x)


Input IDs size: torch.Size([17988, 4096])
Attention Mask size: torch.Size([17988, 4096])
Vocabulary Size: 28820
Max Sequence Length: 4096


In [5]:
model=Sequential()
input_length=4096
xtrain_np=xtrain.numpy()
model.add(Embedding(input_dim=52000,output_dim=100,input_length=input_length))
model.add(LSTM(64))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])
model.fit(xtrain_np,ytrain,epochs=5,batch_size=32)

Epoch 1/5
450/450 [==============================] - 967s 2s/step - loss: 0.0130 - accuracy: 1.0000
Epoch 2/5
450/450 [==============================] - 915s 2s/step - loss: 2.4286e-05 - accuracy: 1.0000
Epoch 3/5
450/450 [==============================] - 935s 2s/step - loss: 1.3413e-05 - accuracy: 1.0000
Epoch 4/5
450/450 [==============================] - 951s 2s/step - loss: 8.6601e-06 - accuracy: 1.0000
Epoch 5/5
450/450 [==============================] - 935s 2s/step - loss: 6.0241e-06 - accuracy: 1.0000


In [7]:
xtest_np=xtest.numpy()
model.evaluate(xtest_np,ytest)


113/113 [==============================] - 45s 396ms/step - loss: 5.0825e-06 - accuracy: 1.0000


[5.082535153633216e-06, 1.0]

In [8]:
model.save("newmodel2.keras")